In [31]:
# lectura de los puntos característicos
#paths a los distintos directorios
import numpy as np
import os.path


pathPoints = "../allPoints"
pathDir = "../d_paths.txt"
pathAligns = "../align"
pathData = "../dataWords"

extensionData = ".jpg.data"
extAlign = ".align"
nframe = "frame"
lossData = "loss.txt"

num_lines = 33000

id = 0


In [32]:
def getContentAlign(path):        
    with open(path) as f:
        lines = [line.split() for line in f]
    
    lines.pop()
    lines.pop(0)
    
    for line in lines:
        line[0] = str( int(line[0])/1000 )
        line[1] = str( int(line[1])/1000 )
        
    return lines   
    

In [33]:
def createMatrix( key, params ):    
    print 'for the \"' + key + '\" file...'
    for line in params:
        start = int(line[0])
        finish = int(line[1])
        word = line[2] 
        global id        
        path = pathPoints + "/" + key + "_" + nframe + "s/" + nframe + str(start) + extensionData
        data_verify = getPoints(path)
        
        if data_verify == []:
            f_loss = open("../" + lossData, "aw+")
            value = f_loss.readline()
            
            if not value:
                value = value.rstrip('\n')              
                
            if not key == value:
                f_loss.write(key + "\n")
            f_loss.close()            
            continue;
        else:   
            all_points = np.array(data_verify)
            
            for i in range(start + 1, finish):
                path = pathPoints + "/" + key + "_" + nframe + "s/" + nframe + str(i) + extensionData            
                data_verify = getPoints(path)
                
                if data_verify == []:
                    print "No se pudo cargar: {0}".format(path)
                    continue;
                parcial = np.array(data_verify)
                all_points = np.concatenate((all_points, parcial) , axis = 1 )
            
            data = np.asarray(all_points)        
            path_save = pathData + "/" + word + "/" + word + "_" + str( id ) + ".csv"           
            np.savetxt( path_save, data, fmt="%d", delimiter=",")
            
    id += 1
    if id%500 == 0:        
        perc = (float(id)/num_lines) * 100
        print '+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++'
        print (str(id) + " de " + str(num_lines) + " archivos procesados... " + str(perc) + "%") 
        print '+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++'
        
    

In [34]:
def getPoints(path):
    point_arr = []
    try:
        f_frame = open(path, 'r')
        points = (line for i, line in enumerate(f_frame) if i>=48)

        for point in points:
            point_arr.append( point.rstrip('\r\n').split(',') )

        for point in point_arr:
            point[0] = int(point[0])
            point[1] = int(point[1])

        f_frame.close()
    except IOError as (errno, strerror):
        print "Error E/S ({0}): {1}, {2}".format(errno, strerror, path)
        point_arr = []
    
    return point_arr


In [35]:
#p_align = pathAligns + '/' + 'bbwd4s' + extAlign

#cont_align = getContentAlign(p_align)
#print cont_align
#createMatrix(line_str, cont_align)

In [ ]:
f_dir = open(pathDir)

for i in range(num_lines):
    line_str = f_dir.readline()
    line_str = line_str.rstrip('\n')    
    
    p_align = pathAligns + '/' + line_str + extAlign
    
    cont_align = getContentAlign(p_align)
    
    createMatrix(line_str, cont_align)
 